In [81]:
# # 1. Uninstall
# !pip uninstall -y dgl torch torchdata numpy

# # 2. Install versi spesifik numpy dulu
# !pip install numpy==1.26.4

# # 3. Install versi Torch + torchvision + torchaudio
# !pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1

# # 4. Install DGL yang sesuai
# !pip install dgl==2.1.0

# # 5. Install library tambahan
# !pip install torchdata==0.6.1 scikit-learn scipy pyyaml tqdm


In [82]:
# # UNINSTALL torchdata (kalau ada), install versi yang cocok
# !pip uninstall -y torchdata

# # Install ulang torchdata yang COCOK dengan torch 2.1.1 (versi 0.6.1 TIDAK COCOK)
# # Install versi torchdata 0.11.0
# !pip install torchdata==0.11.0

# # Install DGL 2.1.0 (cocok dengan torch 2.1.1 dan torchdata 0.11.0)
# !pip install dgl==2.1.0


In [83]:
# # UNINSTALL semua dulu
# # !pip uninstall -y dgl dgl-cu12 torch torchdata torchaudio torchvision



# # PASANG versi yang kompatibel (Python 3.11 dan DGL 2.1.0)
# !pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1
# !pip install torchdata==0.6.1  # << Versi INI yang sudah terbukti cocok dengan torch==2.1.1
# !pip install dgl==2.1.0  # atau dgl-cu12==2.1.0 jika pakai GPU CUDA 12
# # 2. Install versi spesifik numpy dulu
# # !pip install numpy==1.26.4


In [84]:
import torch
import dgl
import torchdata

print("Torch version:", torch.__version__)         # 2.1.1
print("DGL version:", dgl.__version__)             # 2.1.0
print("TorchData version:", torchdata.__version__) # 0.11.0


Torch version: 2.0.1+cu117
DGL version: 2.1.0
TorchData version: 0.6.1


In [85]:
import random
import shutil
from argparse import Namespace
from pathlib import Path
from typing import Optional, Callable

import numpy as np
import torch as th
import torch
import torch.nn as nn
import yaml
from scipy.special import softmax
from sklearn.metrics import f1_score, accuracy_score

In [86]:
# Load masing-masing graph dari file
g1, _ = dgl.load_graphs("graph_bank_0.bin")
g2, _ = dgl.load_graphs("graph_bank_11.bin")
g3, _ = dgl.load_graphs("graph_bank_12.bin")
g4, _ = dgl.load_graphs("graph_bank_20.bin")
g5, _ = dgl.load_graphs("graph_bank_27.bin")

# Gabungkan semua graph ke dalam 1 list
g_list = g1 + g2 + g3 + g4 + g5

# Tentukan jumlah kelas (misalnya binary classification = 2 kelas)
label_dict = {"num_classes": torch.tensor([2])}

# Simpan sebagai 1 file .bin yang sesuai format FedHGN
dgl.save_graphs("aml-hetero_random-edges_5.bin", g_list, labels=label_dict)


In [87]:

def load_configs(args):
    with open(args.config_path) as f:
        configs = yaml.load(f, Loader=yaml.FullLoader)

    dataset_configs = configs["datasets"][args.dataset]
    dataset_cname = dataset_configs.pop("cname")
    dataset_path = dataset_configs.pop("path")
    if dataset_configs["task"] == "node_classification":
        task_literal = "nc"
    else:
        raise ValueError("Unknown task type: {}".format(dataset_configs["task"]))
    model_configs = configs["models"].get(f"{args.model}_{task_literal}", configs["models"][args.model])
    framework_configs = configs["frameworks"].get(f"{args.framework}_{task_literal}",
                                                  configs["frameworks"][args.framework])

    if args.framework == "Central":
        args.split_strategy = "centralized"
        args.num_clients = 1
    if args.framework != "FedHGN":
        args.ablation = None
    assert args.split_strategy in ["centralized", "edges", "etypes"]
    if args.split_strategy in ["edges", "etypes"]:
        args.split_strategy = f"random-{args.split_strategy}"

    all_configs = vars(args) | dataset_configs | model_configs | framework_configs
    all_configs["dataset_cname"] = dataset_cname
    all_configs["dataset_path"] = dataset_path

    return Namespace(**all_configs)


def get_save_path(args, prefix="./saves"):
    save_path = Path(prefix, args.framework if args.ablation is None else f"{args.framework}_{args.ablation}",
                     args.model, f"{args.dataset}_{args.split_strategy}_{args.num_clients}")
    save_path.mkdir(parents=True, exist_ok=True)
    old_saves = [int(str(x.name)) for x in save_path.iterdir() if x.is_dir() and str(x.name).isdigit()]
    if len(old_saves) == 0:
        save_num = 1
    else:
        save_num = max(old_saves) + 1
    save_path = save_path / str(save_num)
    save_path.mkdir()

    # copy config files to the save dir
    shutil.copy("./configs.yaml", save_path)

    return str(save_path)


def set_random_seeds(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    th.manual_seed(seed)


def get_data_dict(data, types):
    if len(types) == 1:
        assert not isinstance(data, dict)
        return {types[0]: data}
    else:
        assert isinstance(data, dict)
        return data


def align_schemas(g_list):
    ntypes = []
    etypes = []
    canonical_etypes = []
    for g in g_list:
        ntypes.extend(g.ntypes)
        etypes.extend(g.etypes)
        canonical_etypes.extend(g.canonical_etypes)
    ntypes = list(set(ntypes))
    etypes = list(set(etypes))
    canonical_etypes = list(set(canonical_etypes))
    return ntypes, etypes, canonical_etypes


def print_results(results: dict[str, float]):
    print("\t".join(results.keys()))
    print("\t".join([f"{v:.4f}" for v in results.values()]))


def save_results(results: dict[str, float], save_path: str):
    save_path = Path(save_path)
    with save_path.joinpath("results.txt").open("w") as f:
        f.write("\t".join(results.keys()) + "\n")
        f.write("\t".join([f"{v:.4f}" for v in results.values()]) + "\n")


def evaluate_node_classification(encoder, decoder, dataloader, target_ntype, return_preds=False):
    results = {}
    logits_list = []
    y_true_list = []

    encoder.eval()
    decoder.eval()
    with th.no_grad():
        for iteration, (input_nodes, output_nodes, blocks) in enumerate(dataloader):
            input_features = get_data_dict(blocks[0].srcdata["x"], blocks[0].srctypes)
            output_labels = get_data_dict(blocks[-1].dstdata["y"], blocks[-1].dsttypes)

            h_dict = encoder(blocks, input_features)
            logits = decoder(h_dict[target_ntype])

            logits_list.append(logits.cpu().numpy())
            y_true_list.append(output_labels[target_ntype].cpu().numpy())

    logits = np.concatenate(logits_list, axis=0)
    y_true = np.concatenate(y_true_list, axis=0)

    y_pred = np.argmax(logits, axis=-1)
    y_score = softmax(logits, axis=-1)

    results["accuracy"] = accuracy_score(y_true, y_pred)
    results["macro-f1"] = f1_score(y_true, y_pred, average="macro")
    results["micro-f1"] = f1_score(y_true, y_pred, average="micro")

    if return_preds:
        return results, y_true, y_pred
    else:
        return results



def load_data(args):
    if args.task == "node_classification":
        if args.dataset in ["AML"]:
            load_path = Path(args.dataset_path, f"{args.dataset_cname}_{args.split_strategy}_{args.num_clients}.bin")
            g_list, label_dict = dgl.load_graphs(str(load_path))

            g_list = [g.long() for g in g_list]
            for g in g_list:
                g.ndata["y"] = g.ndata["label"]
                g.ndata["x"] = g.ndata[dgl.NID]  # dummy node feature, will not be used

            out_dim = label_dict["num_classes"][0].item()

            train_nid_dict_list = [
                {ntype: train_mask.nonzero().flatten() for ntype, train_mask in g.ndata["train_mask"].items()} for g in
                g_list]
            val_nid_dict_list = [
                {ntype: val_mask.nonzero().flatten() for ntype, val_mask in g.ndata["val_mask"].items()} for g in
                g_list]
            test_nid_dict_list = [
                {ntype: test_mask.nonzero().flatten() for ntype, test_mask in g.ndata["test_mask"].items()} for g in
                g_list]
        else:
            raise ValueError("Unknown dataset of task {}: {}".format(args.task, args.dataset))
        return g_list, out_dim, train_nid_dict_list, val_nid_dict_list, test_nid_dict_list
    else:
        raise ValueError("Unknown task: {}".format(args.task))

def evaluate_local_models_individually(clients):
    print("Initial Local Evaluation (Before Fed Training)")
    for i, client in enumerate(clients):
        results = client.local_evaluate(is_test=True)
        print(f"Client {i} initial evaluation:")
        print_results(results)
        print("-" * 40)

class EarlyStopping:
    """Early stops the training if validation score/loss doesn't improve after a given patience."""

    def __init__(self, patience=10, delta=1e-5, mode="score", save_path="checkpoint.pt", verbose=False, ):
        """
        Args:
            patience (int): How long to wait after last time validation score/loss improved.
                            Default: 10
            verbose (bool): If True, prints a message for each validation score/loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.delta = delta
        self.mode = mode
        self.save_path = save_path
        self.verbose = verbose
        self.counter = 0
        self.best_score = -np.inf
        self.early_stop = False

    def __call__(self, quantity: float, model: Optional[nn.Module] = None, callback: Optional[Callable] = None):
        if self.mode == "score":
            score = quantity
        elif self.mode == "loss":
            score = -quantity
        else:
            raise ValueError(f"Invalid mode: {self.mode}")

        if score < self.best_score + self.delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.save_checkpoint(quantity, model, callback)
            self.best_score = score
            self.counter = 0

    def save_checkpoint(self, quantity: float, model: Optional[nn.Module] = None, callback: Optional[Callable] = None):
        """Saves model when validation score/loss improves."""
        if self.verbose:
            if self.mode == "score":
                print(f"Validation score increased ({self.best_score:.6f} --> {quantity:.6f}).  Saving model ...")
            elif self.mode == "loss":
                print(f"Validation loss decreased ({-self.best_score:.6f} --> {quantity:.6f}).  Saving model ...")
            else:
                raise ValueError(f"Invalid mode: {self.mode}")

        if model is not None:
            th.save(model.state_dict(), self.save_path)
        if callback is not None:
            callback(self.save_path)


In [88]:
class NodeClassifier(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, pre_activation=F.relu) -> None:
        super(NodeClassifier, self).__init__()
        self.pre_activation = pre_activation
        self.linear = nn.Linear(in_dim, out_dim)

    def forward(self, x: th.FloatTensor) -> th.FloatTensor:
        return self.linear(self.pre_activation(x)) if self.pre_activation else self.linear(x)

In [89]:

from argparse import Namespace
from collections.abc import Callable
from itertools import chain
from typing import Optional, Union

import dgl
import dgl.nn as dglnn
import torch as th
import torch.nn as nn
import torch.nn.functional as F

class RGCNLayer(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, etypes: list[str], num_bases: int, *, use_weight: bool = True,
                 use_bias: bool = True, activation: Optional[Callable] = None, use_self_loop: bool = False,
                 dropout: float = 0.0) -> None:
        super(RGCNLayer, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.etypes = etypes
        self.num_bases = num_bases
        self.use_weight = use_weight
        self.use_bias = use_bias
        self.activation = activation
        self.use_self_loop = use_self_loop

        self.conv = dglnn.HeteroGraphConv(
            {etype: dglnn.GraphConv(in_dim, out_dim, norm='right', weight=False, bias=False) for etype in etypes})

        # always use bases
        # basis coefficients are defined in class HGNModel
        if self.use_weight:
            self.bases = nn.Parameter(th.Tensor(num_bases, in_dim, out_dim))
            nn.init.xavier_uniform_(self.bases, gain=nn.init.calculate_gain('relu'))

        # bias
        if use_bias:
            self.h_bias = nn.Parameter(th.Tensor(out_dim))
            nn.init.zeros_(self.h_bias)

        # weight for self loop
        if use_self_loop:
            self.loop_weight = nn.Parameter(th.Tensor(in_dim, out_dim))
            nn.init.xavier_uniform_(self.loop_weight, gain=nn.init.calculate_gain('relu'))

        self.dropout = nn.Dropout(dropout)

    def forward(self, g: dgl.DGLHeteroGraph, inputs: dict[str, th.FloatTensor], basis_coeffs: nn.ParameterDict) -> dict[
        str, th.FloatTensor]:
        with g.local_scope():
            if self.use_weight:
                # compute the weight matrices from bases and basis coefficients
                w_dict = {}
                for etype in self.etypes:
                    w_dict[etype] = {
                        "weight": th.matmul(basis_coeffs[etype], self.bases.view(self.num_bases, -1)).view(self.in_dim,
                                                                                                           self.out_dim)}
            else:
                w_dict = {}

            if g.is_block:
                inputs_src = inputs
                inputs_dst = {k: v[:g.number_of_dst_nodes(k)] for k, v in inputs.items()}
            else:
                inputs_src = inputs_dst = inputs

            hs = self.conv(g, inputs, mod_kwargs=w_dict)

            def _apply(ntype, h):
                if self.use_self_loop:
                    h = h + th.matmul(inputs_dst[ntype], self.loop_weight)
                if self.use_bias:
                    h = h + self.h_bias
                if self.activation:
                    h = self.activation(h)
                return self.dropout(h)

            return {ntype: _apply(ntype, h) for ntype, h in hs.items()}


class RGCN(nn.Module):
    def __init__(self, hidden_dim: int, out_dim: int, etypes: list[str], num_bases: int, *, num_hidden_layers: int = 1,
                 dropout: float = 0.0, use_self_loop: bool = False) -> None:
        super(RGCN, self).__init__()
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.etypes = etypes
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers
        self.dropout = dropout
        self.use_self_loop = use_self_loop

        self.layers = nn.ModuleList()
        # i2h
        self.layers.append(RGCNLayer(self.hidden_dim, self.hidden_dim, etypes, self.num_bases, activation=F.relu,
                                     use_self_loop=self.use_self_loop, dropout=self.dropout, use_weight=False))
        # h2h
        for i in range(self.num_hidden_layers):
            self.layers.append(RGCNLayer(self.hidden_dim, self.hidden_dim, etypes, self.num_bases, activation=F.relu,
                                         use_self_loop=self.use_self_loop, dropout=self.dropout))
        # h2o
        self.layers.append(RGCNLayer(self.hidden_dim, self.out_dim, etypes, self.num_bases, activation=None,
                                     use_self_loop=self.use_self_loop))

    def forward(self, g: Union[dgl.DGLHeteroGraph, list], inputs: dict[str, th.FloatTensor],
                basis_coeffs_encoder: nn.ModuleList) -> dict[str, th.FloatTensor]:
        h = inputs
        if isinstance(g, dgl.DGLHeteroGraph):
            # full graph
            for layer, basis_coeffs in zip(self.layers, chain([None], basis_coeffs_encoder)):
                h = layer(g, h, basis_coeffs)
        else:
            # minibatch
            blocks = g
            for layer, block, basis_coeffs in zip(self.layers, blocks, chain([None], basis_coeffs_encoder)):
                h = layer(block, h, basis_coeffs)
        return h


class HGNModel(nn.Module):
    def __init__(self, args: Namespace, out_dim: int, ntypes: list[str], etypes: list[str],
                 canonical_etypes: list[tuple[str, str, str]], num_nodes_dict: dict[str, int]) -> None:
        super(HGNModel, self).__init__()
        self.model_name = args.model
        self.num_bases = args.num_bases
        self.ntypes = ntypes
        self.etypes = etypes
        self.canonical_etypes = canonical_etypes

        # embedding layer: private
        self.embed_layer = dglnn.HeteroEmbedding(num_nodes_dict, args.hidden_dim)
        # self.linear_layer = dglnnHeteroLinear(in_dim_dict, args.hidden_dim)

        # HGNN model: shared
        if self.model_name == "RGCN":
            assert args.num_layers > 1
            # basis coefficients for relations: private
            self.basis_coeffs_encoder = nn.ModuleList()
            for _ in range(args.num_layers - 1):
                param_dict = nn.ParameterDict()
                for etype in self.etypes:
                    param_dict[etype] = nn.Parameter(th.Tensor(self.num_bases))
                    nn.init.xavier_uniform_(param_dict[etype].view(1, -1), gain=nn.init.calculate_gain('relu'))
                self.basis_coeffs_encoder.append(param_dict)
            self.model = RGCN(args.hidden_dim, out_dim, etypes, self.num_bases, num_hidden_layers=args.num_layers - 2,
                              dropout=args.dropout, use_self_loop=args.use_self_loop)
        else:
            raise ValueError(f"Unknown model name {self.model_name}")

    def forward(self, g: Union[dgl.DGLHeteroGraph, list], inputs: dict[str, th.FloatTensor]) -> dict[
        str, th.FloatTensor]:
        # ntype-specific embedding/projection
        if isinstance(g, dgl.DGLHeteroGraph):
            # full graph
            nids_dict = {ntype: g.nodes(ntype) for ntype in g.ntypes}
        else:
            # minibatch
            # g is a list of DGLBlock
            nids_dict = get_data_dict(g[0].srcdata[dgl.NID], g[0].srctypes)
        h_embed_dict = self.embed_layer(nids_dict)
        # h_linear_dict = self.linear_layer(inputs)
        # h_dict = h_embed_dict | h_linear_dict
        h_dict = h_embed_dict

        # HGNN model forward
        h_dict = self.model(g, h_dict, self.basis_coeffs_encoder)

        return h_dict




In [90]:

import random
from argparse import Namespace
from collections import defaultdict
from pathlib import Path
from typing import Optional

import dgl
import torch as th
import torch.nn.functional as F
import tqdm


# FedHGN client
class Client:
    def __init__(self, args: Namespace, data: tuple, client_id: int) -> None:
        self.id = client_id
        self.lr = args.lr
        self.optim = args.optim
        self.weight_decay = args.weight_decay
        self.num_local_epochs = args.num_local_epochs
        self.align_reg = args.align_reg
        self.ablation = args.ablation
        self.task = args.task
        self.device = args.device

        if self.ablation is None:
            # set_others_basis_coeffs
            self.others_basis_coeffs_encoder = None
            self.others_basis_coeffs_decoder = None

        # Use GPU-based neighborhood sampling if possible
        num_workers = 5 if args.device.type == "cpu" else 0
        if self.task == "node_classification":
            g, out_dim, train_nid_dict, val_nid_dict, test_nid_dict = data
            self.g = g.to(self.device)
            self.ntypes = g.ntypes
            self.etypes = list(dict.fromkeys(g.etypes))
            self.canonical_etypes = g.canonical_etypes
            self.out_dim = out_dim
            self.train_nid_dict = {k: v.to(self.device) for k, v in train_nid_dict.items()}
            self.val_nid_dict = {k: v.to(self.device) for k, v in val_nid_dict.items()}
            self.test_nid_dict = {k: v.to(self.device) for k, v in test_nid_dict.items()}
            self.num_nodes_dict = {ntype: g.num_nodes(ntype) for ntype in g.ntypes}
            assert len(self.g.ntypes) == 1 or len(self.g.ndata["y"].keys()) == 1
            assert len(self.train_nid_dict.keys()) == 1
            assert len(self.val_nid_dict.keys()) == 1
            assert len(self.test_nid_dict.keys()) == 1

            self.target_ntype = list(self.train_nid_dict.keys())[0]

            sampler = dgl.dataloading.MultiLayerFullNeighborSampler(args.num_layers)
            self.train_dataloader = dgl.dataloading.DataLoader(self.g, self.train_nid_dict, sampler,
                                                               batch_size=args.batch_size, shuffle=True,
                                                               drop_last=False, num_workers=num_workers,
                                                               device=args.device, use_uva=False)
            self.val_dataloader = dgl.dataloading.DataLoader(self.g, self.val_nid_dict, sampler,
                                                             batch_size=args.batch_size, shuffle=False, drop_last=False,
                                                             num_workers=num_workers, device=args.device, use_uva=False)
            self.test_dataloader = dgl.dataloading.DataLoader(self.g, self.test_nid_dict, sampler,
                                                              batch_size=args.batch_size, shuffle=False,
                                                              drop_last=False, num_workers=num_workers,
                                                              device=args.device, use_uva=False)
            self.encoder = HGNModel(args, args.hidden_dim, self.ntypes, self.etypes, self.canonical_etypes,
                                    self.num_nodes_dict)
            self.encoder.to(args.device)
            self.decoder = NodeClassifier(args.hidden_dim, self.out_dim)
            self.decoder.to(args.device)
        else:
            raise ValueError("Unknown task: {}".format(self.task))

    def local_update(self) -> float:
        if self.optim == "Adam":
            optimizer = th.optim.Adam(list(self.encoder.parameters()) + list(self.decoder.parameters()), lr=self.lr,
                                      weight_decay=self.weight_decay)
        elif self.optim == "SGD":
            optimizer = th.optim.SGD(list(self.encoder.parameters()) + list(self.decoder.parameters()), lr=self.lr,
                                     weight_decay=self.weight_decay)
        else:
            raise ValueError("Unknown optimizer: {}".format(self.optim))

        self.encoder.train()
        self.decoder.train()
        avg_epoch_loss = 0
        with tqdm.tqdm(range(self.num_local_epochs), desc=f"Client {self.id}") as tq:
            if self.task == "node_classification":
                for epoch in tq:
                    epoch_loss = 0
                    num_samples = 0
                    for iteration, (input_nodes, output_nodes, blocks) in enumerate(self.train_dataloader):
                        input_features = get_data_dict(blocks[0].srcdata["x"], blocks[0].srctypes)
                        output_labels = get_data_dict(blocks[-1].dstdata["y"], blocks[-1].dsttypes)

                        h_dict = self.encoder(blocks, input_features)
                        logits = self.decoder(h_dict[self.target_ntype])
                        logp = F.log_softmax(logits, dim=-1)
                        batch_loss = F.nll_loss(logp, output_labels[self.target_ntype])
                        align_reg_term = self.compute_alignment_regularization() * self.align_reg if self.ablation is None else 0

                        optimizer.zero_grad()
                        (batch_loss + align_reg_term).backward()
                        optimizer.step()

                        epoch_loss += batch_loss.item() * logits.shape[0]
                        num_samples += logits.shape[0]
                    epoch_loss /= num_samples
                    # print training info
                    tq.set_postfix({"epoch-loss": f"{epoch_loss:.4f}"}, refresh=False)
                    avg_epoch_loss += epoch_loss
            else:
                raise ValueError("Unknown task: {}".format(self.task))

        avg_epoch_loss /= self.num_local_epochs
        return avg_epoch_loss

    def local_evaluate(self, is_test=False, return_preds=False):
        """Evaluate the model on validation/test set."""
        self.encoder.eval()
        self.decoder.eval()

        with th.no_grad():
            # Get the correct mask based on evaluation mode
            mask_dict = self.test_nid_dict if is_test else self.val_nid_dict
            target_ntype = list(mask_dict.keys())[0]  # Get the first/only node type

            # Get the dataloader for evaluation
            eval_dataloader = self.test_dataloader if is_test else self.val_dataloader

            logits_list = []
            y_true_list = []

            for input_nodes, output_nodes, blocks in eval_dataloader:
                # Get input features and output labels
                input_features = get_data_dict(blocks[0].srcdata["x"], blocks[0].srctypes)
                output_labels = get_data_dict(blocks[-1].dstdata["y"], blocks[-1].dsttypes)

                # Forward pass
                h_dict = self.encoder(blocks, input_features)
                logits = self.decoder(h_dict[target_ntype])

                logits_list.append(logits.cpu())
                y_true_list.append(output_labels[target_ntype].cpu())

            # Concatenate all batches
            logits = th.cat(logits_list, dim=0)
            y_true = th.cat(y_true_list, dim=0)

            # Calculate predictions and metrics
            y_pred = logits.argmax(dim=1)
            acc = (y_true == y_pred).float().mean().item()
            f1_micro = f1_score(y_true, y_pred, average='micro')
            f1_macro = f1_score(y_true, y_pred, average='macro')

            result = {
                "accuracy": acc,
                "micro-f1": f1_micro,
                "macro-f1": f1_macro
            }

            if return_preds:
                return result, y_true.numpy(), y_pred.numpy()
            else:
                return result


    def set_others_basis_coeffs(self, others_basis_coeffs_encoder, others_basis_coeffs_decoder):
        if self.ablation is None:
            self.others_basis_coeffs_encoder = others_basis_coeffs_encoder
            self.others_basis_coeffs_decoder = others_basis_coeffs_decoder
        else:
            raise AssertionError

    def compute_alignment_regularization(self):
        reg = 0
        # encoder coeffs
        local_basis_coeffs_encoder = th.stack(
            [th.stack([param for param in param_dict.values()]) for param_dict in self.encoder.basis_coeffs_encoder])
        diff = local_basis_coeffs_encoder.unsqueeze(2) - self.others_basis_coeffs_encoder.unsqueeze(1)
        min_diff, _ = th.min(th.sum(th.square(diff), dim=-1), dim=-1)
        reg += min_diff.sum()
        # decoder coeffs
        if self.others_basis_coeffs_decoder is not None:
            diff = th.stack([v for v in self.decoder.basis_coeffs_decoder.values()], dim=0).unsqueeze(
                1) - self.others_basis_coeffs_decoder
            min_diff, _ = th.min(th.sum(th.square(diff), dim=-1), dim=-1)
            reg += min_diff.sum()
        return reg


# FedHGN server
class Server:
    def __init__(self, args: Namespace, ntypes: list[str], etypes: list[str],
                 canonical_etypes: list[tuple[str, str, str]], out_dim: Optional[int] = None) -> None:
        self.num_clients = args.num_clients
        self.ablation = args.ablation
        # obtain initial state_dict from a dummy HGNN model
        if self.ablation is None or self.ablation == "B":
            dummy_encoder = HGNModel(args, args.hidden_dim, ["ntype"], ["etype"], [("ntype", "etype", "ntype")],
                                     {"ntype": 1})
        else:
            dummy_encoder = HGNModel(args, args.hidden_dim, ntypes, etypes, canonical_etypes,
                                     {ntype: 1 for ntype in ntypes})
        dummy_encoder.to(args.device)
        state_dict_encoder = dummy_encoder.state_dict()
        # discard the embedding layer
        keys_to_remove = []
        for key in state_dict_encoder:
            if key.startswith("embed_layer"):
                keys_to_remove.append(key)
        # discard things based on ablation type
        if self.ablation is None or self.ablation == "B":
            # discard the basis coefficients
            for key in state_dict_encoder:
                if key.startswith("basis_coeffs_encoder"):
                    keys_to_remove.append(key)
        elif self.ablation == "C":
            # discard the bases
            for key in state_dict_encoder:
                if "bases" in key:
                    keys_to_remove.append(key)
        for key in keys_to_remove:
            del state_dict_encoder[key]
        self.state_dict_encoder = state_dict_encoder

        # obtain initial state_dict from a dummy decoder model
        if args.task == "node_classification":
            assert isinstance(out_dim, int)
            dummy_decoder = NodeClassifier(args.hidden_dim, out_dim)
        else:
            raise ValueError("Unknown task: {}".format(args.task))
        dummy_decoder.to(args.device)
        state_dict_decoder = dummy_decoder.state_dict()
        # discard things based on ablation type
        keys_to_remove = []
        if self.ablation is None or self.ablation == "B":
            # discard the basis coefficients
            for key in state_dict_decoder:
                if key.startswith("basis_coeffs_decoder"):
                    keys_to_remove.append(key)
        elif self.ablation == "C":
            # discard the bases
            for key in state_dict_decoder:
                if "bases" in key:
                    keys_to_remove.append(key)
        for key in keys_to_remove:
            del state_dict_decoder[key]
        self.state_dict_decoder = state_dict_decoder

        if self.ablation is None:
            # set initial collected coefficients as zeros
            self.all_clients_basis_coeffs_encoder = [th.zeros((args.num_layers, 1, args.num_bases), device=args.device)
                                                     for _ in range(self.num_clients)]
            if args.task == "node_classification":
                self.all_clients_basis_coeffs_decoder = None
            else:
                raise ValueError("Unknown task: {}".format(args.task))

    def send_model(self, client: Client) -> None:
        # send state_dict
        client.encoder.load_state_dict(self.state_dict_encoder, strict=False)
        client.decoder.load_state_dict(self.state_dict_decoder, strict=False)
        if self.ablation is None:
            # send basis coefficients
            others_basis_coeffs_encoder = th.cat(
                [self.all_clients_basis_coeffs_encoder[i] for i in range(self.num_clients) if i != client.id], dim=1)
            if self.all_clients_basis_coeffs_decoder is None:
                others_basis_coeffs_decoder = None
            else:
                others_basis_coeffs_decoder = th.cat(
                    [self.all_clients_basis_coeffs_decoder[i] for i in range(self.num_clients) if i != client.id],
                    dim=0)
            client.set_others_basis_coeffs(others_basis_coeffs_encoder, others_basis_coeffs_decoder)

    def aggregate_model(self, clients: list[Client], client_weights: Optional[list[float]] = None) -> None:
        # aggregate state_dict_encoder and state_dict_decoder
        # here we just apply FedAvg to state_dict_encoder and state_dict_decoder
        state_dict_encoder_list = [client.encoder.state_dict() for client in clients]
        state_dict_decoder_list = [client.decoder.state_dict() for client in clients]
        client_weights = [1.0 / len(clients) for _ in range(len(clients))] if client_weights is None else client_weights
        # take care of the missing keys due to different schemas across clients
        for key in self.state_dict_encoder:
            total_weight = 0
            aggregated_param = 0
            for state_dict_encoder, client_weight in zip(state_dict_encoder_list, client_weights):
                if key in state_dict_encoder:
                    aggregated_param += state_dict_encoder[key] * client_weight
                    total_weight += client_weight
            self.state_dict_encoder[key] = aggregated_param / total_weight
        for key in self.state_dict_decoder:
            total_weight = 0
            aggregated_param = 0
            for state_dict_decoder, client_weight in zip(state_dict_decoder_list, client_weights):
                if key in state_dict_decoder:
                    aggregated_param += state_dict_decoder[key] * client_weight
                    total_weight += client_weight
            self.state_dict_decoder[key] = aggregated_param / total_weight

        if self.ablation is None:
            # update basis coefficients at server
            # need to detach()
            for client in clients:
                self.all_clients_basis_coeffs_encoder[client.id] = th.stack(
                    [th.stack([param.detach() for param in param_dict.values()]) for param_dict in
                     client.encoder.basis_coeffs_encoder])
                if self.all_clients_basis_coeffs_decoder is not None:
                    self.all_clients_basis_coeffs_decoder[client.id] = th.stack(
                        [param.detach() for param in client.decoder.basis_coeffs_decoder.values()], dim=0)


class FedHGN:
    def __init__(self, args: Namespace) -> None:
        self.max_rounds = args.max_rounds
        self.num_clients = args.num_clients
        self.fraction = args.fraction
        self.task = args.task
        self.val_interval = args.val_interval
        self.patience = args.patience
        self.save_path = args.save_path
        self.ablation = args.ablation

        if self.task == "node_classification":
            # data preparation for all clients
            g_list, out_dim, train_nid_dict_list, val_nid_dict_list, test_nid_dict_list = load_data(args)
            # align schemas (for ablation)
            ntypes, etypes, canonical_etypes = align_schemas(g_list)
            # setup clients
            self.clients = [
                Client(args, (g_list[i], out_dim, train_nid_dict_list[i], val_nid_dict_list[i], test_nid_dict_list[i]),
                       i) for i in range(self.num_clients)]
            # setup a server
            self.server = Server(args, ntypes, etypes, canonical_etypes, out_dim)
            # client weights for updates and evaluation metrics
            self.train_client_weights = [sum([len(nids) for nids in nid_dict.values()]) for nid_dict in
                                         train_nid_dict_list]
            train_num_samples = sum(self.train_client_weights)
            self.train_client_weights = [weight / train_num_samples for weight in self.train_client_weights]
            self.val_client_weights = [sum([len(nids) for nids in nid_dict.values()]) for nid_dict in val_nid_dict_list]
            val_num_samples = sum(self.val_client_weights)
            self.val_client_weights = [weight / val_num_samples for weight in self.val_client_weights]
            self.test_client_weights = [sum([len(nids) for nids in nid_dict.values()]) for nid_dict in
                                        test_nid_dict_list]
            test_num_samples = sum(self.test_client_weights)
            self.test_client_weights = [weight / test_num_samples for weight in self.test_client_weights]
        else:
            raise ValueError("Unknown task: {}".format(self.task))

    def train(self) -> None:
        sample_size = max(round(self.fraction * self.num_clients), 1)
        early_stopping = EarlyStopping(patience=self.patience, mode="score", save_path=self.save_path, verbose=True)
        with tqdm.tqdm(range(self.max_rounds), desc="FedHGN") as tq:
            for round_no in tq:
                # randomly select clients
                selected_clients = random.sample(self.clients, sample_size)
                # server sends model to clients
                for client in selected_clients:
                    self.server.send_model(client)
                # clients train on their local data (local update)
                round_loss = 0
                for client in selected_clients:
                    client_loss = client.local_update()
                    round_loss += client_loss
                round_loss /= sample_size
                # server aggregates model updates from clients
                selected_clients_weights = [self.train_client_weights[client.id] for client in selected_clients]
                total_weight = sum(selected_clients_weights)
                selected_clients_weights = [weight / total_weight for weight in selected_clients_weights]
                self.server.aggregate_model(selected_clients, selected_clients_weights)

                tq.set_postfix({"round-loss": f"{round_loss:.4f}"}, refresh=False)

                # validation and early stopping
                if (round_no + 1) % self.val_interval == 0:
                    val_results = self.evaluate(is_test=False)
                    print_info = {key: f"{value:.4f}" for key, value in val_results.items()}
                    tq.set_postfix(print_info, refresh=False)
                    if self.task == "node_classification":
                        # quantity = (val_results["macro-f1"] + val_results["micro-f1"]) / 2
                        quantity = val_results["accuracy"]
                    else:
                        raise ValueError("Unknown task: {}".format(self.task))
                    early_stopping(quantity, callback=self.save_checkpoint)
                    if early_stopping.early_stop:
                        print("Early stopping")
                        break

    def evaluate(self, is_test: bool = False) -> dict[str, float]:
        # send the latest model to all clients
        for client in self.clients:
            self.server.send_model(client)
        # get client weights
        client_weights = self.test_client_weights if is_test else self.val_client_weights
        # clients evaluate on their local data
        avg_results = defaultdict(float)
        for i, (client, weight) in enumerate(zip(self.clients, client_weights)):
            client_results = client.local_evaluate(is_test)

            # Tambahan: print per client
            print(f"Client {i} Evaluation ({'Test' if is_test else 'Validation'}):")
            print_results(client_results)
            print("-" * 40)

            for k, v in client_results.items():
                avg_results[k] += v * weight
        return dict(avg_results)



    def save_checkpoint(self, save_path: str) -> None:
        save_path = Path(save_path)
        save_path.mkdir(parents=True, exist_ok=True)
        # save server model
        th.save(self.server.state_dict_encoder, save_path / "server_encoder.pt")
        th.save(self.server.state_dict_decoder, save_path / "server_decoder.pt")
        if self.ablation is None:
            th.save(self.server.all_clients_basis_coeffs_encoder, save_path / "all_clients_basis_coeffs_encoder.pt")
            th.save(self.server.all_clients_basis_coeffs_decoder, save_path / "all_clients_basis_coeffs_decoder.pt")
        # save clients model
        for i, client in enumerate(self.clients):
            th.save(client.encoder.state_dict(), save_path / f"client_{i}_encoder.pt")
            th.save(client.decoder.state_dict(), save_path / f"client_{i}_decoder.pt")

    def load_checkpoint(self, load_path: str) -> None:
        load_path = Path(load_path)
        # load server model
        self.server.state_dict_encoder = th.load(load_path / "server_encoder.pt")
        self.server.state_dict_decoder = th.load(load_path / "server_decoder.pt")
        if self.ablation is None:
            self.server.all_clients_basis_coeffs_encoder = th.load(load_path / "all_clients_basis_coeffs_encoder.pt")
            self.server.all_clients_basis_coeffs_decoder = th.load(load_path / "all_clients_basis_coeffs_decoder.pt")
        # load clients model
        for i, client in enumerate(self.clients):
            client.encoder.load_state_dict(th.load(load_path / f"client_{i}_encoder.pt"))
            client.decoder.load_state_dict(th.load(load_path / f"client_{i}_decoder.pt"))


In [91]:
def main(args):
    global fl_framework  # <-- tambahkan ini supaya bisa diakses dari luar

    if args.framework == "FedHGN":
        fl_framework = FedHGN(args)
    else:
        raise ValueError("Unknown framework.")

    evaluate_local_models_individually(fl_framework.clients)
    fl_framework.train()
    fl_framework.load_checkpoint(args.save_path)
    avg_results = fl_framework.evaluate(is_test=True)
    print_results(avg_results)
    save_results(avg_results, args.save_path)

In [92]:

import os
import warnings

warnings.filterwarnings("ignore")  # Nonaktifkan semua warning Python
os.environ['DGL_WARNINGS'] = '0'   # Nonaktifkan warning dari DGL


In [93]:
# !pip install numpy==1.26.4

In [94]:
import torch as th
from types import SimpleNamespace

# Isi konfigurasi secara manual (ganti sesuai kebutuhan)
args = SimpleNamespace()
args.dataset = "AML"
args.split_strategy = "edges"
args.framework = "FedHGN"  # Bisa juga: FedAvg, FedProx, Local, Central
args.ablation = None       # Atau: "B", "C", "B+C"
args.model = "RGCN"
args.num_clients = 5
args.gpu = -1
args.random_seed = 1000
args.config_path = "configs.yaml"  # Pastikan sesuai path file kamu di Colab

# Load config dan inisialisasi device
args = load_configs(args)
args.device = th.device(f"cuda" if args.gpu >= 0 and th.cuda.is_available() else "cpu")
args.save_path = get_save_path(args)

# Set random seed untuk reprodusibilitas
set_random_seeds(args.random_seed)

# Jalankan federated learning
main(args)


Initial Local Evaluation (Before Fed Training)
Client 0 initial evaluation:
accuracy	micro-f1	macro-f1
0.3274	0.3274	0.2744
----------------------------------------
Client 1 initial evaluation:
accuracy	micro-f1	macro-f1
0.9225	0.9225	0.4892
----------------------------------------
Client 2 initial evaluation:
accuracy	micro-f1	macro-f1
0.7557	0.7557	0.4685
----------------------------------------
Client 3 initial evaluation:
accuracy	micro-f1	macro-f1
0.8261	0.8261	0.4783
----------------------------------------
Client 4 initial evaluation:
accuracy	micro-f1	macro-f1
0.9447	0.9447	0.4858
----------------------------------------


Client 3: 100%|██████████| 3/3 [00:02<00:00,  1.41it/s, epoch-loss=0.5155]

Client 0: 100%|██████████| 3/3 [00:02<00:00,  1.17it/s, epoch-loss=0.4999]

Client 1: 100%|██████████| 3/3 [00:01<00:00,  1.61it/s, epoch-loss=0.4935]

Client 2: 100%|██████████| 3/3 [00:01<00:00,  1.63it/s, epoch-loss=0.4823]

Client 4: 100%|██████████| 3/3 [00:01<00:00,  1.68it/s, epoch-loss=0.4763]


Client 0 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9649	0.9649	0.4911
----------------------------------------
Client 1 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9695	0.9695	0.4923
----------------------------------------
Client 2 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9723	0.9723	0.4930
----------------------------------------
Client 3 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9639	0.9639	0.4908
----------------------------------------


FedHGN:   0%|          | 1/1000 [00:12<3:29:23, 12.58s/it, accuracy=0.9719, micro-f1=0.9719, macro-f1=0.4929]

Client 4 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9895	0.9895	0.4974
----------------------------------------
Validation score increased (-inf --> 0.971892).  Saving model ...


...


Client 0: 100%|██████████| 3/3 [00:01<00:00,  1.66it/s, epoch-loss=0.1828]

Client 2: 100%|██████████| 3/3 [00:02<00:00,  1.47it/s, epoch-loss=0.1622]

Client 1: 100%|██████████| 3/3 [00:02<00:00,  1.36it/s, epoch-loss=0.1721]

Client 3: 100%|██████████| 3/3 [00:02<00:00,  1.08it/s, epoch-loss=0.2015]

Client 4: 100%|██████████| 3/3 [00:02<00:00,  1.22it/s, epoch-loss=0.1390]


Client 0 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9649	0.9649	0.4911
----------------------------------------
Client 1 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9695	0.9695	0.4923
----------------------------------------
Client 2 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9723	0.9723	0.4930
----------------------------------------
Client 3 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9639	0.9639	0.4908
----------------------------------------


FedHGN:   1%|          | 10/1000 [02:19<3:49:22, 13.90s/it, accuracy=0.9719, micro-f1=0.9719, macro-f1=0.4929]

Client 4 Evaluation (Validation):
accuracy	micro-f1	macro-f1
0.9895	0.9895	0.4974
----------------------------------------
EarlyStopping counter: 10 out of 10
Early stopping


Client 0 Evaluation (Test):
accuracy	micro-f1	macro-f1
0.9653	0.9653	0.4912
----------------------------------------
Client 1 Evaluation (Test):
accuracy	micro-f1	macro-f1
0.9694	0.9694	0.4922
----------------------------------------
Client 2 Evaluation (Test):
accuracy	micro-f1	macro-f1
0.9721	0.9721	0.4929
----------------------------------------
Client 3 Evaluation (Test):
accuracy	micro-f1	macro-f1
0.9639	0.9639	0.4908
----------------------------------------
Client 4 Evaluation (Test):
accuracy	micro-f1	macro-f1
0.9895	0.9895	0.4974
----------------------------------------
accuracy	micro-f1	macro-f1
0.9719	0.9719	0.4929
